# Buding an Agent with Tool Use using Strands SDK and MCP
One of the benefits of an agent is its ability to use Tools. Leveraging Tools enables agents to perform complex tasks beyond reasoning capabilities.
When given a task, the agent evaluates the task requirements, then selects an appropriate Tool from its available toolkit, and strategically applies them to achieve specific goals.

This process involves using the LLM to reason about tool selection, understanding how to properly utilize each Tool, and integrating the results back into the agent's workflow.
In addition, agents could also be given access to external knowledge bases, interface with APIs to extend their problem-solving capability that can address a broader range of challenges.

In this lab, we are going to walk through an example of an agent with Tool use.
This use case focuses on answering a user's questions about media assets. Media asset details, such as titles, plot, synopsis are types of information commonly stored and available in the title management systems of media companies. These details could be leveraged by an AI agent to gather the information about the titles based on a user's questions in natural language. While title information is stored in the knowledge base, there may be additional details relevant to titles that are not readily available, for instances, getting the latest movie ratings or reviews about the given title. An agent would need to access other internal / external sources, for instance, IMDB through web APIs to gather such details.

In our lab, we demonstrate calling an API through a Tool that allows the agent to gather extra information. The combination of details from knowledge bases and the API completes the user query.

<img src="../../imgs/lab6-strands-mcp.png" width="800"> 

The steps shown in the above diagram are defined as followed:

1. User initiates a task request. (e.g. Ask for a rating of a movie)
2. An agent received the request, first extracts the media title information from the knowledge base.
3. Based on the user query, the agent retrieves the rating information by invoking a Tool. The Tool makes an API  that provides the rating information for a title given the title_id.
4. The agent receives both the title and rating information for the given title.
5. Agent returns the final response to the user.

In this noteebook, we'll implement the agentic application using [StrandsAgent SDK](https://strandsagents.com/latest/), and MCP server.  

In [ ]:
!pip install strands-agents strands-agents-tools -qU

<div class="alert alert-info"> 💡 The following cell imports Strands agent packages into the kernel. Sometimes, the library might not be updated from the installation. If you run into any issues with importing the following libraries, simply restart the kernel again and rerun the notebook. 
</div>

In [ ]:
from strands import Agent, tool

restore variable for context to be used in this notebook.

In [ ]:
%store -r

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()])

In [ ]:
import boto3
import os
import uuid

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

bedrock_client = boto3.client('bedrock-runtime', region)
os.environ["AWS_REGION"] = region


Create a default AWS profile for this user. We will associate this profile with MCP so that it could authenticate access to the relevant MCP servers.

In [ ]:
region

In [ ]:
credentials = session.get_credentials()
frozen_creds = credentials.get_frozen_credentials()

os.makedirs(f"{os.environ['HOME']}/.aws", exist_ok = True) 
with open(f"{os.environ['HOME']}/.aws/credentials", "w") as f:
    f.write(f"""[default]
aws_access_key_id={frozen_creds.access_key}
aws_secret_access_key={frozen_creds.secret_key}
aws_session_token={frozen_creds.token}
aws_default_region={region}
""")

# Create a Strands Media Agent

We will create a media agent and equip it to have access to an `Amazon Bedrock Knowledge Base` where it retrieves the given title information, and a lambda function to retrieve additional details, such as ratings.

The role of the agent is to answer user questions about movie titles, including synopsis, rating and more.

We will provide the following instructions to our agent:
```
You are a professional media agent. Your task is to help users find the information
related to the media based on the tool and knowledge bases available to you.

You have access to the following tools:
1. knowledge base tool with kb_id of {kb_id} that contains movie / show information including title, year, duration and genre.
2. a rating retrieval tool that provide information about media details, including the title, ratings.


To enable tools for the agent, we will create MCP client that connects to the tools. 

## Setup MCP Server
In the following code block, we'll create an MCP client for each tool that we will provide to the Strand agent. 
1. Lambda MCP Client - an MCP client responsible for invoking Lambda function
2. Knowledge Bases for Bedrock MCP Client - an MCP client respnosible for retrieving information from the given Knowledge Bases for Bedrock.

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to MCP server
lambda_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=["awslabs.lambda-tool-mcp-server@latest"],
        env = {
        "FUNCTION_LIST": "lab6-lambda",
        "AWS_REGION" : region,
        "AWS_PROFILE" : "default"
      }
    )
))

kb_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=["awslabs.bedrock-kb-retrieval-mcp-server@latest"],
        env = {
        "AWS_REGION" : region,
        "AWS_PROFILE" : "default"
      },
      disabled=False
    )
))



## Define Strands Agent
In order to assign tools to the agent, we need to first list all the tools available for the MCP server, and attach them to the agent. In the following block, we enable all the supported tools in these MCP servers, and assign them to the media agent.

To learn more information about this MCP servers, please refer to [knowledge bases for Bedrock MCP sevrer](https://awslabs.github.io/mcp/servers/bedrock-kb-retrieval-mcp-server/) and [AWS Lambda tool MCP server](https://awslabs.github.io/mcp/servers/lambda-tool-mcp-server/). There are many useful MCP servers available to use, follow this [documentation](https://awslabs.github.io/mcp/) for a complete list of all AWS supported MCP servers.

In [ ]:
lab6_kb_id

In [ ]:
# Create an agent with MCP tools
with lambda_mcp_client, kb_mcp_client:
    # Get the tools from the MCP server
    tools = lambda_mcp_client.list_tools_sync() + kb_mcp_client.list_tools_sync()
    print(tools)
    agent = Agent(tools=tools, 
                  model="us.anthropic.claude-3-5-haiku-20241022-v1:0", 
                  system_prompt=f"""You are a professional media agent. Your task is to help users find the information
related to the media based on the tool and knowledge bases available to you.

You have access to the following tools:
1. knowledge base tool with kb_id of {lab6_kb_id} that contains movie / show information including title, year, duration and genre.
2. a rating retrieval tool that provide information about media details, including the title, ratings. The tool accept title_id as input.

If you need to retrieve the title_id from the knowledge base, look for the title_id column for the value.
For example, a retrieved media contains the following data:

title_id: 123
title: Some title
year: 2025
duration: 100 minutes
""",
callback_handler=None)

# Testing the Agent
With all the components in place, let's test out our agent. We'll start by asking some questions about specific titles. 

In [ ]:
with lambda_mcp_client, kb_mcp_client:
    output = agent("Tell me the genre of 'Quantum Shadows'?")

Prints out the output

In [ ]:
print(output.message['content'][0]['text'])

Try another question related to the previous question. Strands agent keeps tracks of the conversation history for the session. In the following example, we asked about the rating of the movie without providing the name or title information about the show. The Agent gathered the information from the previous conversation in the message history

In [ ]:
with lambda_mcp_client, kb_mcp_client:
    output = agent("What is the rating this movie?")

Prints out the output

In [ ]:
print(output.message['content'][0]['text'])

# Clean Up

In [ ]:
os.remove(f"{os.environ['HOME']}/.aws/credentials")

# Conclusion
In this lab, we demonstrated how to build a Strands Agent as an AI assistant that helps users find media information efficiently. We first created a Knowledge Base for Bedrock, which contains the media details made available to the agent to find media title information. We use the [Knowledge Bases for Bedrock MCP server](https://awslabs.github.io/mcp/servers/bedrock-kb-retrieval-mcp-server/) to connect to the knowledge base. Additionally, we connected to a [lambda MCP server](https://awslabs.github.io/mcp/servers/lambda-tool-mcp-server/) to the agent which provides real time media rating from an external system via an API call. Using these tools, the AI agent helps users answer relevant questions about the media assets in their media library / archive. 